**aaa**

In [122]:
import pandas as pd
from datetime import datetime

In [123]:
# Load Datasets
df_analytics = pd.read_csv("datasets/extras/archivo_combinado_cleaned.csv")
df_images = pd.read_csv("datasets/imagedataFINALmarch2025.csv")

In [124]:
df_images.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 394 entries, 0 to 393
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   nombre_archivo    394 non-null    object
 1   fecha             394 non-null    object
 2   objeto_principal  394 non-null    object
 3   color_dominante   394 non-null    object
 4   emocion_primaria  394 non-null    object
 5   categoría         393 non-null    object
dtypes: object(6)
memory usage: 18.6+ KB


In [125]:
df_analytics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702 entries, 0 to 701
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   identificador_de_la_publicacion  702 non-null    int64  
 1   identificador_de_la_cuenta       702 non-null    int64  
 2   nombre_de_usuario_de_la_cuenta   702 non-null    object 
 3   nombre_de_la_cuenta              702 non-null    object 
 4   descripcion                      302 non-null    object 
 5   duracion_segundos                702 non-null    int64  
 6   hora_de_publicacion              702 non-null    object 
 7   enlace_permanente                13 non-null     object 
 8   tipo_de_publicacion              702 non-null    object 
 9   comentario_sobre_los_datos       0 non-null      float64
 10  fecha                            702 non-null    object 
 11  alcance                          642 non-null    float64
 12  me_gusta              

In [126]:
 #Step 2. Prepare the dataset for analysis
df_interim_analytics = (
    df_analytics
    .copy()
    .set_axis(
        df_analytics.columns.str.replace(' ', '_') # reemplazar espacio por _
        .str.replace('á', 'a') # reemplazar
        .str.replace('é', 'e') # reemplazar
        .str.replace('í', 'i') # reemplazar
        .str.replace('ó', 'o') # reemplazar
        .str.replace('ú', 'u') # reemplazar
        .str.replace(r'\W','',regex=True) # reemplazar caracteres no alfabéticos por nada
        .str.lower() # el resultado lo pasamos a minúsculas
        .str.slice(0,40), axis=1, # slice sirve para cortar el texto a un máximo de 40 caracteres
    )
    

)
df_interim_analytics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702 entries, 0 to 701
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   identificador_de_la_publicacion  702 non-null    int64  
 1   identificador_de_la_cuenta       702 non-null    int64  
 2   nombre_de_usuario_de_la_cuenta   702 non-null    object 
 3   nombre_de_la_cuenta              702 non-null    object 
 4   descripcion                      302 non-null    object 
 5   duracion_segundos                702 non-null    int64  
 6   hora_de_publicacion              702 non-null    object 
 7   enlace_permanente                13 non-null     object 
 8   tipo_de_publicacion              702 non-null    object 
 9   comentario_sobre_los_datos       0 non-null      float64
 10  fecha                            702 non-null    object 
 11  alcance                          642 non-null    float64
 12  me_gusta              

In [127]:
df_interim_analytics.drop("identificador_de_la_publicacion", axis=1, inplace=True)
df_interim_analytics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702 entries, 0 to 701
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   identificador_de_la_cuenta      702 non-null    int64  
 1   nombre_de_usuario_de_la_cuenta  702 non-null    object 
 2   nombre_de_la_cuenta             702 non-null    object 
 3   descripcion                     302 non-null    object 
 4   duracion_segundos               702 non-null    int64  
 5   hora_de_publicacion             702 non-null    object 
 6   enlace_permanente               13 non-null     object 
 7   tipo_de_publicacion             702 non-null    object 
 8   comentario_sobre_los_datos      0 non-null      float64
 9   fecha                           702 non-null    object 
 10  alcance                         642 non-null    float64
 11  me_gusta                        702 non-null    int64  
 12  veces_que_se_compartio          702 

In [128]:
df_interim_analytics.drop("identificador_de_la_cuenta", axis=1, inplace=True)
df_interim_analytics.drop("nombre_de_usuario_de_la_cuenta", axis=1, inplace=True)
df_interim_analytics.drop("nombre_de_la_cuenta", axis=1, inplace=True)
##df_interim_analytics.drop("descripcion", axis=1, inplace=True)
df_interim_analytics.drop("enlace_permanente", axis=1, inplace=True)
df_interim_analytics.drop("tipo_de_publicacion", axis=1, inplace=True)
df_interim_analytics.drop("comentario_sobre_los_datos", axis=1, inplace=True)
df_interim_analytics.drop("fecha", axis=1, inplace=True)

df_interim_analytics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702 entries, 0 to 701
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   descripcion             302 non-null    object 
 1   duracion_segundos       702 non-null    int64  
 2   hora_de_publicacion     702 non-null    object 
 3   alcance                 642 non-null    float64
 4   me_gusta                702 non-null    int64  
 5   veces_que_se_compartio  702 non-null    int64  
 6   visitas_al_perfil       527 non-null    float64
 7   respuestas              702 non-null    int64  
 8   navegacion              637 non-null    float64
 9   clics_en_el_enlace      25 non-null     float64
 10  seguimientos            24 non-null     float64
 11  toques_en_stickers      24 non-null     float64
 12  visualizaciones         123 non-null    float64
 13  sentimiento             702 non-null    float64
dtypes: float64(8), int64(4), object(2)
memory 

In [129]:
# 2.2 Create the final dataset with the target in front, show result
df_final_analytics = (
    df_interim_analytics
    .copy()
    .reindex(
        columns=(
            ['hora_de_publicacion'] +
            [c for c in df_interim_analytics.columns.to_list() if c not in ['hora_de_publicacion']]
        )
    )
)
df_final_analytics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702 entries, 0 to 701
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   hora_de_publicacion     702 non-null    object 
 1   descripcion             302 non-null    object 
 2   duracion_segundos       702 non-null    int64  
 3   alcance                 642 non-null    float64
 4   me_gusta                702 non-null    int64  
 5   veces_que_se_compartio  702 non-null    int64  
 6   visitas_al_perfil       527 non-null    float64
 7   respuestas              702 non-null    int64  
 8   navegacion              637 non-null    float64
 9   clics_en_el_enlace      25 non-null     float64
 10  seguimientos            24 non-null     float64
 11  toques_en_stickers      24 non-null     float64
 12  visualizaciones         123 non-null    float64
 13  sentimiento             702 non-null    float64
dtypes: float64(8), int64(4), object(2)
memory 

In [130]:
# Renombrar la columna 'sentimiento' a 'sentimiento_score'
df_final_analytics.rename(columns={'sentimiento': 'sentimiento_score'}, inplace=True)

# Mostrar el DataFrame con la columna renombrada
print(df_final_analytics.head())

   hora_de_publicacion                             descripcion  \
0  2023-01-01 15:32:00  En la recta final del Guadalupe -Reyes   
1  2023-01-01 16:17:00                                     NaN   
2  2023-01-01 18:34:00        Faltan 303 días \npara Halloween   
3  2023-01-03 18:36:00                            Para el frío   
4  2023-01-03 20:16:00                                     NaN   

   duracion_segundos  alcance  me_gusta  veces_que_se_compartio  \
0                  0      NaN         3                       0   
1                  0      NaN         4                       0   
2                  0      NaN         5                       0   
3                  0      NaN         7                       0   
4                 13      NaN         3                       0   

   visitas_al_perfil  respuestas  navegacion  clics_en_el_enlace  \
0                NaN           0         NaN                 NaN   
1                NaN           0         NaN                 NaN

In [131]:
# Eliminar las filas en las que la columna 'duracion_segundos' sea mayor a cero porque son videos y no los tengo en el dataset de imagenes
df_final_analytics = df_final_analytics[df_final_analytics['duracion_segundos'] <= 0]

# Eliminar la columna 'duracion_segundos'
df_final_analytics.drop('duracion_segundos', axis=1, inplace=True)
df_final_analytics.info()



<class 'pandas.core.frame.DataFrame'>
Index: 385 entries, 0 to 701
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   hora_de_publicacion     385 non-null    object 
 1   descripcion             185 non-null    object 
 2   alcance                 359 non-null    float64
 3   me_gusta                385 non-null    int64  
 4   veces_que_se_compartio  385 non-null    int64  
 5   visitas_al_perfil       302 non-null    float64
 6   respuestas              385 non-null    int64  
 7   navegacion              357 non-null    float64
 8   clics_en_el_enlace      10 non-null     float64
 9   seguimientos            10 non-null     float64
 10  toques_en_stickers      16 non-null     float64
 11  visualizaciones         91 non-null     float64
 12  sentimiento_score       385 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 42.1+ KB


In [132]:
# Mostrar el DataFrame resultante
df_final_analytics.head()

,hora_de_publicacion,descripcion,alcance,me_gusta,veces_que_se_compartio,visitas_al_perfil,respuestas,navegacion,clics_en_el_enlace,seguimientos,toques_en_stickers,visualizaciones,sentimiento_score
0,2023-01-01 15:32:00,En la recta final del Guadalupe -Reyes,NaN,3,0,NaN,0,NaN,NaN,NaN,NaN,NaN,0.0
1,2023-01-01 16:17:00,NaN,NaN,4,0,NaN,0,NaN,NaN,NaN,NaN,NaN,-2.0
2,2023-01-01 18:34:00,Faltan 303 días \npara Halloween,NaN,5,0,NaN,0,NaN,NaN,NaN,NaN,NaN,0.0
3,2023-01-03 18:36:00,Para el frío,NaN,7,0,NaN,0,NaN,NaN,NaN,NaN,NaN,0.0
7,2023-01-06 12:38:00,NaN,NaN,6,0,NaN,0,NaN,NaN,NaN,NaN,NaN,-2.0


In [134]:
df_final_analytics.to_csv('datasets/extras/final_analytics.csv', index=False)

In [133]:
df_images.head()

,nombre_archivo,fecha,objeto_principal,color_dominante,emocion_primaria,categoría
0,IMG_20250120_150630_215.jpg,2025-01-20,person,#595e67,happy,Selfie
1,IMG_20250117_112006_771.jpg,2025-01-17,person,#5c616b,happy,"Selfie, Comida y Bebida"
2,IMG_20250113_093221_483.jpg,2025-01-13,coffee_mug,#838593,happy,"Selfie, Comida y Bebida"
3,IMG_20250105_124127_869.jpg,2025-01-05,coffee_mug,#858387,happy,"Selfie, Comida y Bebida"
4,IMG_20250103_151034_782.jpg,2025-01-03,hall,#535a77,neutral,Selfie


In [97]:
# Crear una función para convertir fechas y manejar formatos mixtos
def convertir_fecha(fecha):
    for fmt in ('%d/%m/%Y %H:%M', '%m/%d/%Y %H:%M'):
        try:
            return pd.to_datetime(fecha, format=fmt)
        except ValueError:
            continue
    return pd.NaT

# Aplicar la función de conversión a cada valor en la columna 'Hora de publicación'
df_final_analytics['hora_de_publicacion'] = df_final_analytics['hora_de_publicacion'].apply(convertir_fecha)

# Crear una columna 'fecha' basada en 'Hora de publicación'
df_final_analytics['fecha'] = df_final_analytics['hora_de_publicacion'].dt.date

# Asegurarse de que la columna 'fecha' sea del tipo datetime64[ns]
df_final_analytics['fecha'] = pd.to_datetime(df_final_analytics['fecha'], errors='coerce')

# Verificar los tipos de las columnas
print(df_final_analytics.info())

<class 'pandas.core.frame.DataFrame'>
Index: 385 entries, 0 to 701
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   hora_de_publicacion     385 non-null    datetime64[ns]
 1   alcance                 359 non-null    float64       
 2   me_gusta                385 non-null    int64         
 3   veces_que_se_compartio  385 non-null    int64         
 4   visitas_al_perfil       302 non-null    float64       
 5   respuestas              385 non-null    int64         
 6   navegacion              357 non-null    float64       
 7   clics_en_el_enlace      10 non-null     float64       
 8   seguimientos            10 non-null     float64       
 9   toques_en_stickers      16 non-null     float64       
 10  visualizaciones         91 non-null     float64       
 11  sentimiento_score       385 non-null    float64       
 12  fecha                   385 non-null    datetime64[ns]


In [98]:
df_final_analytics.head()

,hora_de_publicacion,alcance,me_gusta,veces_que_se_compartio,visitas_al_perfil,respuestas,navegacion,clics_en_el_enlace,seguimientos,toques_en_stickers,visualizaciones,sentimiento_score,fecha
0,2023-01-10 10:04:00,1989.0,7,0,9.0,0,2019.0,NaN,NaN,NaN,NaN,0.0,2023-01-10
2,2023-12-31 10:17:00,750.0,4,0,3.0,0,762.0,NaN,NaN,NaN,NaN,0.0,2023-12-31
3,2023-12-29 13:44:00,555.0,8,0,NaN,2,620.0,76.0,NaN,NaN,NaN,-2.0,2023-12-29
4,2023-12-29 10:09:00,841.0,3,0,3.0,0,840.0,NaN,NaN,NaN,NaN,0.0,2023-12-29
11,2023-12-28 07:21:00,401.0,1,0,4.0,0,389.0,NaN,NaN,NaN,NaN,-2.0,2023-12-28


In [99]:
df_final_analytics.to_csv('datasets/final_analytics.csv', index=False)

Hasta aquí está correcto, el resto lo haré manual

In [51]:
# Asegurarse de que la columna 'fecha' sea del tipo datetime64[ns]
df_images['fecha'] = pd.to_datetime(df_images['fecha'], errors='coerce')
df_images.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 394 entries, 0 to 393
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   nombre_archivo    394 non-null    object        
 1   fecha             394 non-null    datetime64[ns]
 2   objeto_principal  394 non-null    object        
 3   color_dominante   394 non-null    object        
 4   emocion_primaria  394 non-null    object        
 5   categoría         393 non-null    object        
dtypes: datetime64[ns](1), object(5)
memory usage: 18.6+ KB


In [56]:
#Verificar duplicados
print(df_final_analytics[df_final_analytics.duplicated(subset='fecha', keep=False)])
print(df_images[df_images.duplicated(subset='fecha', keep=False)])

    hora_de_publicacion  alcance  me_gusta  veces_que_se_compartio  \
3   2023-12-29 13:44:00    555.0         8                       0   
4   2023-12-29 10:09:00    841.0         3                       0   
20  2023-12-25 13:12:00   1560.0         7                       0   
21  2023-12-25 09:13:00    828.0         6                       1   
25  2023-12-23 09:05:00    336.0         2                       0   
..                  ...      ...       ...                     ...   
668 2024-10-02 14:29:00    356.0         3                       0   
669 2024-10-02 14:29:00    576.0         4                       0   
687 2024-03-01 07:18:00    188.0         1                       0   
690 2024-03-01 07:18:00    207.0         1                       0   
694 2024-03-01 06:00:00    512.0         5                       1   

     visitas_al_perfil  respuestas  navegacion  clics_en_el_enlace  \
3                  NaN           2       620.0                76.0   
4                  

In [57]:
# Eliminar duplicados
df_final_analytics = df_final_analytics.drop_duplicates(subset='fecha')
df_images = df_images.drop_duplicates(subset='fecha')

In [58]:
df_final_analytics.info()

<class 'pandas.core.frame.DataFrame'>
Index: 235 entries, 0 to 701
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   hora_de_publicacion     235 non-null    datetime64[ns]
 1   alcance                 223 non-null    float64       
 2   me_gusta                235 non-null    int64         
 3   veces_que_se_compartio  235 non-null    int64         
 4   visitas_al_perfil       203 non-null    float64       
 5   respuestas              235 non-null    int64         
 6   navegacion              221 non-null    float64       
 7   clics_en_el_enlace      10 non-null     float64       
 8   seguimientos            8 non-null      float64       
 9   toques_en_stickers      10 non-null     float64       
 10  visualizaciones         71 non-null     float64       
 11  sentimiento_score       235 non-null    float64       
 12  fecha                   235 non-null    datetime64[ns]


In [59]:
df_images.info()

<class 'pandas.core.frame.DataFrame'>
Index: 239 entries, 0 to 392
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   nombre_archivo    239 non-null    object        
 1   fecha             239 non-null    datetime64[ns]
 2   objeto_principal  239 non-null    object        
 3   color_dominante   239 non-null    object        
 4   emocion_primaria  239 non-null    object        
 5   categoría         238 non-null    object        
dtypes: datetime64[ns](1), object(5)
memory usage: 13.1+ KB


In [60]:
# Combinar los DataFrames basados en la columna 'fecha' y eliminar registros sin coincidencias
df_combinado = df_final_analytics.merge(df_images, on='fecha', how='inner')

# Mostrar el DataFrame combinado
print(df_combinado.head())


  hora_de_publicacion  alcance  me_gusta  veces_que_se_compartio  \
0 2023-12-31 10:17:00    750.0         4                       0   
1 2023-12-29 13:44:00    555.0         8                       0   
2 2023-12-28 07:21:00    401.0         1                       0   
3 2023-12-27 15:06:00   8231.0         4                       0   
4 2023-12-25 13:12:00   1560.0         7                       0   

   visitas_al_perfil  respuestas  navegacion  clics_en_el_enlace  \
0                3.0           0       762.0                 NaN   
1                NaN           2       620.0                76.0   
2                4.0           0       389.0                 NaN   
3               62.0           0      8064.0                 NaN   
4                9.0           0      1546.0                 NaN   

   seguimientos  toques_en_stickers  visualizaciones  sentimiento_score  \
0           NaN                 NaN              NaN                0.0   
1           NaN                 

In [61]:
df_combinado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   hora_de_publicacion     164 non-null    datetime64[ns]
 1   alcance                 156 non-null    float64       
 2   me_gusta                164 non-null    int64         
 3   veces_que_se_compartio  164 non-null    int64         
 4   visitas_al_perfil       140 non-null    float64       
 5   respuestas              164 non-null    int64         
 6   navegacion              155 non-null    float64       
 7   clics_en_el_enlace      8 non-null      float64       
 8   seguimientos            5 non-null      float64       
 9   toques_en_stickers      5 non-null      float64       
 10  visualizaciones         44 non-null     float64       
 11  sentimiento_score       164 non-null    float64       
 12  fecha                   164 non-null    datetime64

In [54]:
# Descubrí que algunas entradas se duplicaron, entonces voy a borrarlas

# Guardar el DataFrame combinado en un nuevo archivo CSV
df_combinado.to_csv('datasets/final_datset_march2025.csv', index=False)

In [55]:
df_combinado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   hora_de_publicacion     636 non-null    datetime64[ns]
 1   alcance                 608 non-null    float64       
 2   me_gusta                636 non-null    int64         
 3   veces_que_se_compartio  636 non-null    int64         
 4   visitas_al_perfil       438 non-null    float64       
 5   respuestas              636 non-null    int64         
 6   navegacion              607 non-null    float64       
 7   clics_en_el_enlace      20 non-null     float64       
 8   seguimientos            7 non-null      float64       
 9   toques_en_stickers      19 non-null     float64       
 10  visualizaciones         69 non-null     float64       
 11  sentimiento_score       636 non-null    float64       
 12  fecha                   636 non-null    datetime64